<a href="https://colab.research.google.com/github/dikemahoko/Authentic/blob/master/MUZAMANI_LEE_TADIWA_TSRWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading the dataset zip file
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip


--2025-09-17 21:19:03--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276294756 (263M) [application/zip]
Saving to: ‘GTSRB_Final_Training_Images.zip’

GTSRB_Final_Trainin 100%[===================>] 263.50M  5.78MB/s    in 72s     

2025-09-17 21:20:16 (3.64 MB/s) - ‘GTSRB_Final_Training_Images.zip’ saved [276294756/276294756]



In [3]:

!unzip -q GTSRB_Final_Training_Images.zip

replace GTSRB/Final_Training/Images/00000/00000_00000.ppm? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


In [4]:
!ls

GTSRB  GTSRB_Final_Training_Images.zip	sample_data


In [5]:
!apt-get install -y tree


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (97.4 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [6]:
!tree -L 3 GTSRB

GTSRB
├── Final_Training
│   └── Images
│       ├── 00000
│       ├── 00001
│       ├── 00002
│       ├── 00003
│       ├── 00004
│       ├── 00005
│       ├── 00006
│       ├── 00007
│       ├── 00008
│       ├── 00009
│       ├── 00010
│       ├── 00011
│       ├── 00012
│       ├── 00013
│       ├── 00014
│       ├── 00015
│       ├── 00016
│       ├── 00017
│       ├── 00018
│       ├── 00019
│       ├── 00020
│       ├── 00021
│       ├── 00022
│       ├── 00023
│       ├── 00024
│       ├── 00025
│       ├── 00026
│       ├── 00027
│       ├── 00028
│       ├── 00029
│       ├── 00030
│       ├── 00031
│       ├── 00032
│       ├── 00033
│       ├── 00034
│       ├── 00035
│       ├── 00036
│       ├── 00037
│       ├── 00038
│       ├── 00039
│       ├── 00040
│       ├── 00041
│       └── 00042
└── Readme-Images.txt

45 directories, 1 file


In [8]:
# Importing and Data Loading
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


# The path
data_dir = '/content/GTSRB'
IMG_HEIGHT = 32
IMG_WIDTH = 32
NUM_CATEGORIES = 43

# --- Load Data Function ---
def load_data(data_dir):
    images = []
    labels = []
    for category in range(NUM_CATEGORIES):
        category_path = os.path.join(data_dir, 'Final_Training', 'Images', str(category).zfill(5))
        for img_name in os.listdir(category_path):
            if img_name.endswith('.ppm'):
                img_path = os.path.join(category_path, img_name)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
                    images.append(img)
                    labels.append(category)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# --- Execute Loading and Pre-processing ---
images, labels = load_data(data_dir)
images = images / 255.0  # Normalize
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, NUM_CATEGORIES)
y_val = to_categorical(y_val, NUM_CATEGORIES)

print(f"Dataset loaded. Training samples: {len(X_train)}")

Dataset loaded. Training samples: 31367


In [10]:
# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    fill_mode="nearest"
)

In [ ]:
#Building and Training the Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CATEGORIES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
history = model.fit(
    data_generator.flow(X_train, y_train, batch_size=64),
    epochs=25,
    validation_data=(X_val, y_val)
)

# Saving the full Keras model first
model.save('traffic_sign_model.h5')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


491/491 ━━━━━━━━━━━━━━━━━━━━ 71s 141ms/step - accuracy: 0.2084 - loss: 2.9282 - val_accuracy: 0.6686 - val_loss: 1.1701
Epoch 2/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 65s 132ms/step - accuracy: 0.5658 - loss: 1.3589 - val_accuracy: 0.8689 - val_loss: 0.5076
Epoch 3/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 62s 126ms/step - accuracy: 0.7085 - loss: 0.8982 - val_accuracy: 0.9192 - val_loss: 0.3043
Epoch 4/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 85s 133ms/step - accuracy: 0.7705 - loss: 0.6974 - val_accuracy: 0.9477 - val_loss: 0.2035
Epoch 5/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 63s 128ms/step - accuracy: 0.8184 - loss: 0.5614 - val_accuracy: 0.9654 - val_loss: 0.1417
Epoch 6/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 64s 130ms/step - accuracy: 0.8402 - loss: 0.4890 - val_accuracy: 0.9730 - val_loss: 0.1141
Epoch 7/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 65s 131ms/step - accuracy: 0.8578 - loss: 0.4261 - val_accuracy: 0.9806 - val_loss: 0.0916
Epoch 8/25
491/491 ━━━━━━━━━━━━━━━━━━━━ 62s 126ms/step - accuracy: 0.8813 - loss: 0.3677 - val

In [12]:
#Converting to TFLite
# Loading the saved Keras model
keras_model = tf.keras.models.load_model('traffic_sign_model.h5')


converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model saved as model.tflite")

Saved artifact at '/tmp/tmpggf1xb66'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 43), dtype=tf.float32, name=None)
Captures:
  137912212806288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212802832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212815120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212802448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212814160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212811856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212804368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137912212815312: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved as model.tflite


In [13]:
# Download

from google.colab import files

files.download('model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>